## Ancho de banda de un video

- ¿Cúanto pesa una imagen RGB de 1920x1080?

<center>1920 1080 3 size(uint8) = 49.766.400 b ~ 50 Mb</center>

- Tradicionalmente un video es una secuencia de imágenes a 24 cuadros por segundo (fps)
- ¿Cuánto ancho de banda se necesita para ver una película en tiempo real?

<center>50Mb  24 fps = 1200 Mb/s = 1.2 Gb/s</center>

- Este ancho de banda es infactible. ¿Qué necesitamos para que un servicio de streaming pueda funcionar?

https://toolstud.io/video/filesize.php

# Compresión de datos

- Codificar la información usando menos "bits" que la representación original
- La compresión puede ser de tipo
    - *Lossless* (sin pérdidas): Los datos originales pueden reconstruirse perfectamente
    - *Lossy* (con pérdidas): Se reconstruye una versión aproximada de los datos originales
- Recordemos el modelo de Shannon

In [ ]:
G = nx.DiGraph(); G.graph['rankdir'] = 'LR'; G.graph['dpi'] = 120
G.add_node(0, shape='rect', label='Fuente', color='black')
G.add_node(1, shape='rect', label='Transmisor')
G.add_node(2, shape='rect', label='Canal')
G.add_node(3, shape='rect', label='Receptor')
G.add_node(4, shape='rect', label='Destino')
G.add_node(5, shape='rect', style='filled', fillcolor='pink', label='Ruido')
G.add_edge(0, 1, style='solid', label='mensaje')
G.add_edge(1, 2, style='dashed', label='señal')
G.add_edge(2, 3, style='dashed', label='señal')
G.add_edge(3, 4, style='solid', label='mensaje')
G.add_edge(5, 2, style='dashed',)
draw(G , show='ipynb')

El siguiente es el modelo general del transmisor en el esquema anterior

In [ ]:
G = nx.DiGraph(); G.graph['rankdir'] = 'LR'; G.graph['dpi'] = 120;
G.add_node(0, shape='rect', label='Transformación', color='black')
G.add_node(1, shape='rect', label='Cuantización')
G.add_node(2, shape='rect', label='Codificación de fuente')
G.add_node(3, shape='rect', label='Codificación de canal', color='grey')
G.add_edge(0, 1, style='solid'); G.add_edge(1, 2, style='solid'); 
G.add_edge(2, 3, style='solid'); draw(G , show='ipynb')

- **Transformación:** Cambia la representación de los datos para eliminar redundancia y reducir correlaciones. Ejemplos: PCM, DFT, PCA y DCT. Usualmente se aplica sobre bloques de datos
- **Cuantización:** Escoge un número fijo de valores representativos (diccionario), es decir acorta el "largo de palabra" de los datos. Ejemplos: cuantización escalar y cuantización vectorial (VQ)
    - Largo de palabra: Cantidad de bits necesaria para representar un símbolo de código
    - Diccionario o alfabeto: Conjunto de símbolos o palabras
- **Codificación de fuente:** Convierte el diccionario fijo del cuantizador en un código de largo variable, que es más eficiente de transmitir. Ejemplos: codificación de Huffman
- **Codificación de canal:** Se encarga de "robustecer" el resultado anterior para que sobreviva la transmisión digital. Ejemplo: dígito verificador para detectar y corregir errores

## Compresión de imágenes

- Hemos aprendido que una imagen se puede proyectar al espacio de frecuencias sin pérdida de información (transformación invertible)
- Sabemos que las imágenes tienen alta correlación entre píxeles inmediatamente vecinos, es decir hay gran redundancia
- Por ende el espectro concentra gran parte de la información en la frecuencia central
- La DFT tiene la desventaja es que incluso si la imagen es real el espectro resultante es complejo
- Una transformación frecuencial alternativa y ampliamente utilizada es la **Discrete Cosine Transform** (DCT)
- La DCT se aplica en bloques contiguos de 8x8 píxeles (JPEG)

## Joint Photographic Experts Group (JPEG)
- Formato ampliamente usado para distribuir imágenes digitales
- Explota las siguientes características del sistema visual humano
    - Somos más sensibles a la iluminación que al color
    - Somos menos sensibles a los componentes de alta frecuencia
- Es un ejemplo de método de compresión con pérdidas para imágenes (lossy)
- Más componentes descartados, mayor compresión, y peor la calidad


1. Preliminar: RGB a YCbCr, tipicamente se hace downsampling 4:2:2
1. Transformación con DCT en bloques de 8x8 sin traslape
1. Cuantización escalar
1. Codificación de Huffman


### Espacio YCbCr
- Y es la luminancia
- Cb y Cr son los componentes cromáticos azul y rojo, respectivamente

$$
Y = K_R R + K_G G + K_B B \\
C_b = 0.5 \frac{B - Y}{1 - K_B} \\
C_R = 0.5 \frac{R - Y}{1 - K_R}
$$
donde $K_R + K_G + K_B = 1$.


In [ ]:
# ITU-R BT.601 
dogo_color = plt.imread('images/lobo.jpg')
Y = np.dot(dogo_color, [0.299, 0.587, 0.114])
plt.matshow(Y, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Y')
C_b = np.dot(dogo_color, [-0.168736, -0.3312, 0.5])
plt.matshow(C_b, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Cb')
C_r = np.dot(dogo_color, [0.5, -0.4186, -0.0813])
plt.matshow(C_r, cmap=plt.cm.Greys_r); plt.axis('off'); plt.title('Cr');

In [ ]:
def f(q_Y, q_C):
    Y = np.round(np.dot(dogo_color, [0.299, 0.587, 0.114])*q_Y/255.0)*255.0/q_Y
    C_b = 128+np.round(np.dot(dogo_color, [-0.168736, -0.3312, 0.5])*q_C/255.0)*255.0/q_C
    C_r = 128+np.round(np.dot(dogo_color, [0.5, -0.4186, -0.0813])*q_C/255.0)*255.0/q_C
    rgb = np.zeros(shape=(Y.shape[0], Y.shape[1], 3))
    rgb[:,:,0] = Y + 1.402 * (C_r-128)
    rgb[:,:,1] = Y - .34414 * (C_b-128) - .71414 * (C_r-128)
    rgb[:,:,2] = Y + 1.772 * (C_b-128)
    np.putmask(rgb, rgb > 255, 255);
    np.putmask(rgb, rgb < 0, 0);
    plt.figure(figsize=(16, 8))
    plt.imshow(np.uint8(rgb)); plt.axis('off'); flush_figures(); 
   
interact(f, q_Y=IntSlider(min=1, max=255, value=255, description="Y levels", layout=slider_layout),
         q_C=IntSlider(min=1, max=255, value=255, description="C levels", layout=slider_layout));

## Discrete Cosine Transform (DCT)

Sea una señal discreta y bidimensional $g[n_1, n_2]$ con índices $n_1 \in [0, N_1-1]$ y $n_2 \in [0, N_2-1]$ su DCT es 

$$
G_C[k_1, k_2] = \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} 4 g[n_1, n_2] \cos \left ( \frac{\pi k_1}{2N_1}(2n_1+1)  \right) \cos \left ( \frac{\pi k_2}{2N_2}(2n_2+1)  \right) 
$$
y su inversa

$$
g[n_1, n_2] = \frac{1}{N_1 N_2}\sum_{k_1=0}^{N_1-1} \sum_{k_2=0}^{N_2-1} w[k_1]w[k_2]G[k_1, k_2] \cos \left ( \frac{\pi k_1}{2N_1}(2n_1+1)  \right) \cos \left ( \frac{\pi k_2}{2N_2}(2n_2+1)  \right), 
$$

donde 
$$
w[k] =\begin{cases}
1/2 & \text{ssi} ~~ k=0\\
1 & \text{ssi} ~~ k \neq 0
\end{cases} 
$$

- La DCT es lineal, y cumple el principio de conservación de energía
- La DCT 2D se puede descomponer en 2 aplicaciones de la DCT 1D
- Notar que La convolución en espacio original no es multiplicación en el espacio DCT!
- La DCT es equivalente a la DFT de una señal "simetricamente extendida"
$$
y[k] =\begin{cases}
x[k] & \text{ssi} ~~ k<N\\
x[2N-1-k] & \text{ssi} ~~ N \leq k < 2N - 1 
\end{cases} 
$$
- Es decir que podemos usar el algoritmo FFT para calcular eficientemente la DCT

La base de la DCT:

In [ ]:
x = np.arange(0, 32, step=1)
X, Y = np.meshgrid(x, x)
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(8, 8)
for n in range(8):
    for m in range(8):
        ax = plt.subplot(gs[n, m])
        ax.matshow(np.cos(np.pi*(2*X+1)*m/(2*len(x)))*np.cos(np.pi*(2*Y+1)*n/(2*len(x))), 
                   cmap=plt.cm.RdBu_r, vmin=-1, vmax=1)
        ax.axis('off')
gs.tight_layout(fig, pad=0.1)

### Ejemplo: Transformación de una imagen usando DCT en bloques de 8x8

In [ ]:
img_seadoge = np.dot(plt.imread('images/lobo.jpg'), [0.299, 0.587, 0.114])  
plt.figure(figsize=(10, 7))
plt.imshow(img_seadoge, cmap=plt.cm.Greys_r);

In [ ]:
DCT2 = lambda g: fftpack.dct( fftpack.dct( g, axis=0, norm='ortho' ), axis=1, norm='ortho' )
IDCT2 = lambda G: fftpack.idct( fftpack.idct( G, axis=0, norm='ortho' ), axis=1, norm='ortho' )

imsize = img_seadoge.shape
dct_matrix = np.zeros(shape=imsize)
dft_matrix = np.zeros(shape=imsize, dtype=np.complex128)

for i in range(0, imsize[0], 8):
    for j in range(0, imsize[1], 8):
        dct_matrix[i:(i+8),j:(j+8)] = DCT2( img_seadoge[i:(i+8),j:(j+8)] )
        dft_matrix[i:(i+8),j:(j+8)] = fftpack.fft2( img_seadoge[i:(i+8),j:(j+8)] ) 

¿Cómo se ven uno a uno la DCT de los bloques?

In [ ]:
def f(block_tile=1):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(img_seadoge[block_tile:block_tile+8, block_tile:block_tile+8],
              cmap=plt.cm.Greys_r)
    ax.set_title("%d 8x8 image block" %(block_tile)); 
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(dct_matrix[block_tile:block_tile+8, block_tile:block_tile+8],
              cmap=plt.cm.Greys_r, 
              extent=[0,np.pi,np.pi,0])
    ax.set_title("8x8 DCT"); 
    plt.tight_layout(); flush_figures(); 

interact(f, block_tile=IntSlider(min=0, max=100, value=0, description="Block tile", layout=slider_layout));

In [ ]:
plt.figure(figsize=(16, 9))
plt.imshow(dct_matrix, cmap=plt.cm.Greys_r, vmin=0.0, vmax=0.01*np.amax(dct_matrix))

Comparemos ahora la DFT y DCT en términos de reconstrucción con pérdidas

In [ ]:
def f(percent):
    im_dct, im_dft = np.zeros(imsize), np.zeros(imsize)
    I = np.unravel_index(np.argsort(np.absolute(dct_matrix), axis=None)[::-1], 
                         dct_matrix.shape)
    dct_thresh = dct_matrix.copy()
    dct_thresh[I[0][int(600*1000*percent/100):], 
               I[1][int(600*1000*percent/100):]] = 0
    I = np.unravel_index(np.argsort(np.absolute(dft_matrix), axis=None)[::-1], 
                         dft_matrix.shape)
    dft_thresh = dft_matrix.copy()
    dft_thresh[I[0][int(600*1000*percent/100):], 
               I[1][int(600*1000*percent/100):]] = 0
    for i in range(0, imsize[0], 8):
        for j in range(0, imsize[1], 8):
            im_dct[i:(i+8),j:(j+8)] = IDCT2( dct_thresh[i:(i+8),j:(j+8)] )
            im_dft[i:(i+8),j:(j+8)] = np.real(fftpack.ifft2( dft_thresh[i:(i+8),j:(j+8)] ))
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(2, 2, 1)
    ax.imshow(im_dft, cmap=plt.cm.Greys_r); ax.axis('off')
    ax.set_title("DFT")
    ax = fig.add_subplot(2, 2, 3)
    ax.imshow(im_dft[200:400, 200:600], cmap=plt.cm.Greys_r); ax.axis('off')
    ax = fig.add_subplot(2, 2, 2)    
    ax.imshow(im_dct, cmap=plt.cm.Greys_r); ax.axis('off')
    ax.set_title("DCT")
    ax = fig.add_subplot(2, 2, 4)
    ax.imshow(im_dct[200:400, 200:600], cmap=plt.cm.Greys_r); ax.axis('off')
    plt.tight_layout(pad=0.1); flush_figures(); 
interact(f, percent=FloatSlider(min=0, max=10, step=0.01, value=100, 
                                  description="Block tile", layout=slider_layout));

In [ ]:
img_seadogo_sym = np.hstack((img_seadoge, np.fliplr(img_seadoge)))
img_seadogo_sym = np.vstack((np.flipud(img_seadogo_sym), img_seadogo_sym))
plt.figure(figsize=(16, 9))
plt.imshow(img_seadogo_sym, cmap=plt.cm.Greys_r);

- DCT y efectos de borde
- optimalidad en termines de aproximacion a transformacion KL
- optimalidad en terminos de señal markoviana con correlacion positiva

## Cuantización escalar

- Es el proceso que realiza la compresión de los datos
- Es una operación de redondeo/truncamiento
- Se define por su número de niveles $L$, niveles de decisión $d_i$ y niveles de representación $r_i$

\begin{equation}
    Q(x)=
    \begin{cases}
      r_1, & d_0 < x \leq d_1 \\
      \vdots & \vdots \\
      r_i, & d_{i-1} < x \leq d_i \\
      \vdots & \vdots \\
      r_L, & d_{L-1} < x \leq d_L
    \end{cases}
\end{equation}

- Cuantización uniforme: $r_i = \frac{1}{2} (d_i + d_i+1)$ y $d_i - d_{i-1} = \Delta ~\forall i$
- Distorción de cuantización: $d_c = |Q(x) -x| $

## Cuantización vectorial

- Extensión del caso anterior a múltiples dimensiones
- Para un set de datos N-dimensionales $X=\{x_1, x_2, \ldots, x_N\}$ se busca un set de prototipos $C=\{c_1, c_2, \ldots, x_L\}$ con $L < N$
- La función de cuantización es $Q(x) = c_k$ donde $k = \text{arg}\min_i \|x - c_i\|$
- Hay que definir una métrica de distancia


1. Inicializar con prototipos aleatorios
2. Asignar los datos a su prototipo más cercano
3. Actualizar los prototipos como el valor medio de sus datos asignados
4. Iterar hasta converger

<center><img src="images/kmeans.gif" width="600"></center>

In [ ]:
from scipy.cluster.vq import vq
from sklearn.cluster import KMeans
img_color = plt.imread("images/InsInformatica.jpg")
img_pixels = np.reshape(img_color, (img_color.shape[0]*img_color.shape[1], 3))
kmeans = KMeans(n_clusters=10).fit(np.float32(img_pixels)) 
centroids = kmeans.cluster_centers_
qnt,_ = vq(np.float32(img_pixels), centroids)
centers_idx = np.reshape(qnt, (img_color.shape[0], img_color.shape[1]))
clustered = np.uint8(centroids[centers_idx])

plt.figure(figsize=((16, 9)))
plt.subplot(121); plt.imshow(img_color); plt.axis('off');
plt.subplot(122); plt.imshow(clustered); plt.axis('off');

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=((10, 9)))
ax = fig.gca(projection='3d')
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c=centroids/255.,s=100);

- En el estándar JPEG se cuantizan los bloques de 8x8 componentes DCT 
- El nivel de los componentes es redondeado usando una matriz de cuantización Q
- La matriz fue diseñada de tal forma que los componentes de alta frecuencia se cuantizan en menos niveles
- El bloque cuantizado se obtiene como $\text{ROUND}(\frac{G_C}{Q})$

In [ ]:
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],[12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],[14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],[24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],[72, 92, 95, 98, 112, 100, 103, 99]])
def f(block_tile=1):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(img_seadoge[block_tile:block_tile+8, block_tile:block_tile+8],
              cmap=plt.cm.Greys_r)
    ax.set_title("%d 8x8 image block" %(block_tile)); 
    ax = fig.add_subplot(1, 3, 2)
    dct_block = dct_matrix[block_tile:block_tile+8, block_tile:block_tile+8]
    ax.imshow(dct_block, cmap=plt.cm.Greys_r, extent=[0,np.pi,np.pi,0])
    ax.set_title("8x8 DCT %d nonzero" %(np.count_nonzero(dct_block))); 
    ax = fig.add_subplot(1, 3, 3)
    quant = np.round(dct_matrix[block_tile:block_tile+8, block_tile:block_tile+8]/Q)
    ax.imshow(quant, cmap=plt.cm.Greys_r, extent=[0,np.pi,np.pi,0])
    ax.set_title("8x8 Quantized %d nonzero" %(np.count_nonzero(quant))); 
    plt.tight_layout(); flush_figures(); 

interact(f, block_tile=IntSlider(min=0, max=100, value=0, description="Block tile", layout=slider_layout));

In [ ]:
area = img_seadoge.shape[0]*img_seadoge.shape[1]
def f(percent):
    im_dct = np.zeros(imsize)
    nnz = np.zeros(dct_matrix.shape)
    if (percent < 50):
        S = 5000/percent
    else:
        S = 200 - 2*percent 
    Q_dyn = np.floor((S*Q + 50) / 100);
    Q_dyn[Q_dyn == 0] = 1
    for i in range(0, imsize[0], 8):
        for j in range(0, imsize[1], 8):
            quant = np.round(dct_matrix[i:(i+8),j:(j+8)]/Q_dyn) 
            im_dct[i:(i+8),j:(j+8)] = IDCT2(quant)
            nnz[i, j] = np.count_nonzero(quant)
    
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(img_seadoge, cmap=plt.cm.Greys_r); ax.axis('off'); plt.title("%0.2f MB" %(area*8/1e+6))
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(im_dct, cmap=plt.cm.Greys_r); ax.axis('off');  plt.title("%0.2f MB" %(np.sum(nnz)*8/1e+6))
    plt.tight_layout(pad=0.1); flush_figures(); 
interact(f, percent=FloatSlider(min=0, max=100, step=0.01, value=100, 
                                  description="Block tile", layout=slider_layout));

### Código de largo fijo y de largo variable

- Codificación de fuente es el proceso de asignar los mensajes a un conjunto de símbolos o códigos. El largo de las "palabras de código" puede ser fijo o variable.
- **Ejemplo:** Digamos que queremos codificar digitalmente las letras del alfabeto (27) usando dos símbolos (código binario)
    - Si usamos largo fijo necesitamos al menos 5 bits ($2^5 = 32$)
- Pero, nosotros sabemos que algunas letras se ocupan más que otras, *e.g.* las vocales. 
    - Si codificamos las letras con más probabilidad de ocurrencia usando menos bits podemos reducir la cantidad de bits transmitidos
    - Es imperativo que los símbolos puedan ser decodificados sin ambiguedad. **Ejemplo:** Sea A = 1 y B = 11, ¿Qué dice aquí?: 111

# Teoría de la información

- Estudio de la cuantificación y transmisión de la información propuesto por Claude Shannon en 1948
- Nos permite medir la cantidad de información y por ende evaluar los resultados de la compresión
- **Entropía de Shannon**. Sea una fuente que emite M mensajes cada uno con probabilidad de ocurrencia $p_i$ se define 

$$
H(X) = \mathbb{E} \left [ \frac{1}{p(X)} \right ]= - \sum_{i=1}^M p_i \log_2 (p_i)  \text{[bits]},
$$
como la entropía, incerteza o información promedio de la fuente. La información de cada mensaje es $-\log_2(p_i)$

- La entropía es siempre positiva y acotada $0 \leq H(X) \leq \log_2(M)$

### Ejemplo: Entropía de lanzar una moneda

Omitiendo la probabilidad de caer de canto podemos modelar la variable aleatoria del resultado de lanzamiento de moneda como Bernoulli 
$$
f(p) = p^k (1-p)^{1-k} ~~ k \in \{0,1\}
$$
donde p es la probabilidad de que salga cara y 1-p la probabilidad de que salga cruz.

Luego la entropía es
$$
H(X) = - p \log(p) - (1-p) \log(1-p)
$$

- ¿Qué significa que la entropía sea mínima? ¿Y que sea máxima?

In [ ]:
p = np.linspace(0, 1, num=100)
plogp = lambda p: p*np.log2(p+1e-10)
H = -plogp(p) - plogp(1-p)
plt.figure(figsize=(12, 4))
plt.xlabel('Bernoulli p')
plt.ylabel('Bernoulli RV entropy')
plt.grid(); plt.plot(p, H);

 ### Ejemplo: código de ancho variable
 
 Sea una canal con 4 mensajes posibles $m_i$ con probabilidades $p = \{1/2, 1/4, 1/8, 1/8\}$.
 
 La información por símbolo es $-\log p = {1, 2, 3, 3}$
 
 Luego una codificación óptima sería $m_1 = 0$, $m_2 = 10$, $m_3 = 110$ y $m_4 = 111$
 
 La información promedio del canal es 1.75 [bits]
 
 <center><img src="images/dendogram.png" width="600"></center>

- El algoritmo de codificación anterior tiene forma de árbol en base 2
- Los mensajes codificados están en las hojas del árbol
- Este es un ejemplo de **código préfijo**, donde ningún código puede ser prefijo de otro. De esta manera se garantiza decodificación sin ambiguedad

## Teorema de codificación de fuente (source coding theorem)

El largo de palabra promedio $\widehat L$ de un algoritmo de codificación binario prefijo satisface
$$
H(X) \leq \widehat L \leq H(X) + 1
$$
Este teorema justifica la definición de entropía como meduda de la cantidad de información/incerteza

## Codificación de Huffman

1. Se estima la probabilidad $p_i$ de cada símbolo
1. Se ordenan los símbolos en orden descendente según $p_i$
1. Juntar los dos con probabilidad menor en un grupo, su probabilidad se suma
1. Volver al paso 2 hasta que queden dos grupos
1. Asignarle 0 y 1 a las ramas izquierda y derecha del árbol, respectivamente

<center><img src="images/huff.gif" width="600"></center>

Ref: http://www.skylondaworks.com/sc_huff.htm

## Teorema de Shannon-Hartley

- Sea un canal con ancho de banda B [Hz] (rango de frecuencias que un canal puede transmitir) y potencia de señal S [W] y potencia del ruido N [W] (aditivo blanco gaussiano), entonces su capacidad es
$$
C = B \log_2 \left(1 + \frac{S}{N} \right) \text{[bits/s]}
$$
- Si la velocidad de transmisión de un canal es R [bits/s] y R < C entonces la probabilidad de errores de comunicación tiende a cero.
- Las limitaciones de un sistema de comunicación son **Ancho de banda** y el ruido **Ruido**

## H.264/MPEG-4

- Aplicación de la DCT para comprimir a través de varios frames

Futuras iteraciones: espectrograma, wavelets, producto tiempo frecuencia